# --- Section 1: Imports ---


In [97]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt

# --- Section 2: Load and Filter Data ---

In [98]:
# --- Section 2: Load and Filter Data ---

def load_and_filter_data(file_path, sheet_name, start_date, end_date):
    """
    Load Excel data, parse sheet, and filter based on date range.
    """
    xls = pd.ExcelFile(file_path)
    df = xls.parse(sheet_name)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    return df


# Load Stock Data for J/K Strategy
file_path = r'C:\Users\garrv\Desktop\JK_Strategy\Monthly_Log_Returns(prcnt) 2002-2024.xlsx'
custom_start_date = pd.to_datetime("2002-01-01")
custom_end_date = pd.to_datetime("2024-12-31")
stock_df = load_and_filter_data(file_path, 'stocks Monthly return', custom_start_date, custom_end_date)

#Load NIFTY Data for Benchmark Comparison
nifty_df = load_and_filter_data(file_path, 'nifty monthly returns', custom_start_date, custom_end_date)

# --- Section 3: J/K Momentum Strategy Implementation ---


In [99]:

def run_jk_strategy(df, J, K, custom_end_date):
    """
    Run J/K momentum strategy and return list of winner/loser portfolios.
    """
    filtered_portfolios = []

    for i in range(len(df) - J - K + 1):
        formation_start = df.iloc[i]['Date']
        formation_end = df.iloc[i + J - 1]['Date']
        holding_start = df.iloc[i + J]['Date']
        holding_end = df.iloc[i + J + K - 1]['Date']

        if holding_end > custom_end_date:
            break

        formation_period = df.iloc[i:i + J, 1:]
        holding_period = df.iloc[i + J:i + J + K, 1:]

        valid_stocks = formation_period.columns[
            ~formation_period.isna().any() & ~holding_period.isna().any()
        ]

        cumulative_returns = (
            1 + formation_period[valid_stocks] / 100).prod() - 1

        if len(cumulative_returns) >= 50:
            top_50_stocks = cumulative_returns.nlargest(50).sort_values()
            deciles = [top_50_stocks.iloc[i:i + 5].index.tolist()
                       for i in range(0, 50, 5)]

            filtered_portfolios.append({
                'Formation Start': formation_start,
                'Formation End': formation_end,
                'Holding Start': holding_start,
                'Holding End': holding_end,
                'Loser Portfolio': deciles[0],
                'Winner Portfolio': deciles[-1]
            })

    return filtered_portfolios

# --- Section 4: Monthly Return and Spread Calculation ---

In [100]:

def calculate_monthly_returns(df, portfolios):
    """
    Calculate monthly average returns for winner and loser portfolios.
    """
    all_dates = df['Date'].unique()
    winner_returns, loser_returns = {}, {}

    for date in all_dates:
        active = [p for p in portfolios if p['Holding Start']
                  <= date <= p['Holding End']]
        monthly = df[df['Date'] == date].set_index('Date')
        win_ret, lose_ret = [], []

        for p in active:
            if set(p['Winner Portfolio']).issubset(monthly.columns):
                win_ret.append(
                    monthly[p['Winner Portfolio']].mean(axis=1).values[0])
            if set(p['Loser Portfolio']).issubset(monthly.columns):
                lose_ret.append(
                    monthly[p['Loser Portfolio']].mean(axis=1).values[0])

        if win_ret:
            winner_returns[date] = sum(win_ret) / len(win_ret)
        if lose_ret:
            loser_returns[date] = sum(lose_ret) / len(lose_ret)

    final_df = pd.DataFrame(list(winner_returns.items()), columns=['Date', 'Winner Portfolio Avg Return']).merge(
        pd.DataFrame(list(loser_returns.items()), columns=['Date', 'Loser Portfolio Avg Return']), on='Date')

    return final_df

# --- Section 5: Return Summary and Display ---


In [101]:
# --- Section 5: Return Summary and Display ---

def calculate_and_display_summary(final_df, J, K):
    """
    Calculate and print average winner, loser, and spread returns.
    """
    avg_winner = final_df['Winner Portfolio Avg Return'].mean()
    avg_loser = final_df['Loser Portfolio Avg Return'].mean()
    spread = avg_winner - avg_loser

    summary = {
        "J (Formation Period)": J,
        "K (Holding Period)": K,
        "Average Winner Return (%)": round(avg_winner, 4),
        "Average Loser Return (%)": round(avg_loser, 4),
        "Spread (Winner - Loser) (%)": round(spread, 4)
    }

    # print("\n=== J/K Strategy Summary ===")
    # for key, value in summary.items():
    #     print(f"{key}: {value}")

    return summary

# --- Section 6: Export to Excel ---


In [102]:
# --- Section 6: Export to Excel ---

def save_results_to_excel(final_df, J, K, start_date, end_date):
    """
    Save the result DataFrame to an Excel file.
    """
    filename = f"monthly_returns_J{J}_K{K}_custom_{
        start_date.strftime('%Y%m%d')}_to_{end_date.strftime('%Y%m%d')}.xlsx"
    final_df.to_excel(filename, index=False, engine='openpyxl')
    print(f"Results saved to {filename}")

In [103]:
# import numpy as np
# import matplotlib.pyplot as plt


# def plot_cumulative_returns_with_nifty(final_df, nifty_df):
#     """
#     Plot cumulative returns for Winner-Loser spread and NIFTY 50.
#     """

#     # Convert percentage log returns to decimal
#     spread_log_ret = (final_df['Winner Portfolio Avg Return'] -
#                       # Spread in decimal
#                       final_df['Loser Portfolio Avg Return']) / 100
#     # Correct column for NIFTY log returns
#     nifty_log_ret = nifty_df['Return deimal']

#     # Calculate cumulative returns
#     cum_spread = np.exp(spread_log_ret.cumsum())
#     cum_nifty = np.exp(nifty_log_ret.cumsum())

#     # Plot
#     plt.figure(figsize=(14, 7))
#     plt.plot(final_df['Date'], cum_spread,
#              label='Winner - Loser (Spread)', linewidth=2, color='green')
#     plt.plot(nifty_df['Date'], cum_nifty,
#              label='NIFTY 50 Index', linewidth=2, color='black')
#     plt.title("Cumulative Returns: Winner-Loser (Spread) vs. NIFTY 50", fontsize=14)
#     plt.xlabel("Date", fontsize=12)
#     plt.ylabel("Cumulative Return (Growth of $1)", fontsize=12)
#     plt.legend(fontsize=12)
#     plt.grid(True)
#     plt.show()

In [119]:
import itertools
import pandas as pd

# --- Final Run: Compact Version ---

# Define J and K lists
J_list, K_list = [1,3,6,9,12], [1,3,6,9,12]

# Generate combinations and initialize summary list
jk_pairs = list(itertools.product(J_list, K_list))
summary_matrix = []

# Run strategy, calculate returns, and store summary
for J, K in jk_pairs:
    portfolios = run_jk_strategy(stock_df, J, K, custom_end_date)
    if not portfolios:
        print(f"Skipping J={J}, K={K}: No valid portfolios.")
        continue
    final_df = calculate_monthly_returns(stock_df, portfolios)
    avg_winner, avg_loser = final_df['Winner Portfolio Avg Return'].mean(
    ), final_df['Loser Portfolio Avg Return'].mean()
    spread = avg_winner - avg_loser
    summary_matrix.append([int(J), int(K), round(avg_winner, 4), round(
        avg_loser, 4), round(spread, 4)])  # Ensure J and K are integers here

# If results exist, create DataFrame and print max spread
if summary_matrix:
    summary_df = pd.DataFrame(summary_matrix, columns=[
        'J', 'K', 'Avg Winner Return (%)', 'Avg Loser Return (%)', 'Spread (Winner - Loser) (%)'
    ])

    # Display summary DataFrame
    display(summary_df)

    # Find and print max spread row
    max_row = summary_df.loc[summary_df['Spread (Winner - Loser) (%)'].idxmax()]
    print("\nMaximum Spread Result:")
    # Ensure J and K are printed as integers
    print(f"J = {int(max_row['J'])}, K = {int(max_row['K'])}")
    print(f"Average Winner Return (%): {max_row['Avg Winner Return (%)']}")
    print(f"Average Loser Return (%): {max_row['Avg Loser Return (%)']}")
    print(
        f"Spread (Winner - Loser) (%): {max_row['Spread (Winner - Loser) (%)']}")
else:
    print("\nNo valid results to display.")

,J,K,Avg Winner Return (%),Avg Loser Return (%),Spread (Winner - Loser) (%)
0,1,1,1.1978,1.4865,-0.2886
1,1,3,1.1438,1.5935,-0.4497
2,1,6,1.3365,1.4954,-0.1589
3,1,9,1.3592,1.5064,-0.1472
4,1,12,1.3356,1.4047,-0.0691
5,3,1,1.5171,1.3042,0.2128
6,3,3,1.5443,1.3044,0.2399
7,3,6,1.7031,1.2686,0.4345
8,3,9,1.6490,1.2674,0.3816
9,3,12,1.6315,1.2835,0.3480



Maximum Spread Result:
J = 6, K = 3
Average Winner Return (%): 1.945
Average Loser Return (%): 1.056
Spread (Winner - Loser) (%): 0.889
